In [13]:
import numpy as np
import time
from datetime import datetime

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper

# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Estimator

# --- Qiskit Circuit Library Import ---
from qiskit.circuit.library import QAOAAnsatz


# ----- Configuration -----
BASIS_SET = "sto-3g"
BOND_LENGTH = 0.7414  # Angstrom
QAOA_REPS = 10         # QAOA/VQE repetitions (p-value)

# ----- Main Calculation Function -----
def main():
    """
    Calculates the H2 ground state energy using VQE with a QAOAAnsatz
    on the FULL 4-qubit Hamiltonian (no Z2 tapering).
    """
    print(f"H₂ Ground State Calculation for Basis: {BASIS_SET}")
    print(f"Using VQE with QAOAAnsatz (p={QAOA_REPS})")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Define the molecule's geometry as a string
    atom_str = f"H 0 0 0; H 0 0 {BOND_LENGTH}"

    # --- 1. Set up the Chemical Problem ---
    start_time = time.time()
    
    # Set up the chemical problem for Qiskit
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET)
    problem = driver.run()
    
    # --- 2. Map Hamiltonian to Qubits (NO Symmetries) ---
    mapper = JordanWignerMapper()
    
    # Map the Hamiltonian directly. This will be 4 qubits.
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    print(f"Running on full {qubit_op.num_qubits}-qubit Hamiltonian (Z2 symmetries NOT applied).")
    print("-" * 50)
    
    # --- 3. Set up VQE components ---
    optimizer = COBYLA(maxiter=1000)
    estimator = Estimator()
    
    # Create the QAOAAnsatz circuit.
    ansatz = QAOAAnsatz(cost_operator=qubit_op, reps=QAOA_REPS)
    
    # QAOAAnsatz requires 2*reps parameters
    init_point = np.random.random(2 * QAOA_REPS) * 0.1
    
    # Move 'initial_point' into the VQE constructor
    vqe_instance = VQE(estimator,
                         ansatz,
                         optimizer,
                         initial_point=init_point) 
    
    # --- 4. Run the Algorithm ---
    print("Running VQE...")
    result_vqe = vqe_instance.compute_minimum_eigenvalue(
        qubit_op
    )
    
    # --- 5. Interpret and Display Results ---
    # --- THIS IS THE FIX ---
    # The mapper is not needed for a non-tapered result.
    final_result = problem.interpret(result_vqe)
    final_energy = final_result.total_energies[0]
    
    print(f"\nCalculation finished in {time.time() - start_time:.2f}s\n")
    print("="*50)
    print("           Final Energy Summary")
    print("="*50)
    print(f"  VQE (QAOAAnsatz, p={QAOA_REPS}) Energy: {final_energy} Hartree")
    # For reference, the exact (FCI) energy is -1.13727017
    print(f"  Reference FCI (Exact) Energy: -1.137270174660903 Hartree")
    print("-" * 50)
    print(f"  Error (VQE vs FCI):  {abs(final_energy - (-1.137270174660903))} Hartree")
    print("="*50)


if __name__ == "__main__":
    main()

H₂ Ground State Calculation for Basis: sto-3g
Using VQE with QAOAAnsatz (p=10)
Started at: 2025-11-03 10:48:45

Running on full 4-qubit Hamiltonian (Z2 symmetries NOT applied).
--------------------------------------------------
Running VQE...


/tmp/ipykernel_25874/2910923180.py:54: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()



Calculation finished in 100.76s

           Final Energy Summary
  VQE (QAOAAnsatz, p=10) Energy: -1.117973251347932 Hartree
  Reference FCI (Exact) Energy: -1.137270174660903 Hartree
--------------------------------------------------
  Error (VQE vs FCI):  0.01929692331297117 Hartree


In [10]:
import numpy as np
import time
from datetime import datetime

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
# --- FIX: Import UCCSD and HartreeFock ---
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Estimator
# --- We no longer need QAOAAnsatz ---
# from qiskit.circuit.library import QAOAAnsatz


# ----- Configuration -----
BASIS_SET = "sto-3g"
BOND_LENGTH = 0.7414  # Angstrom
# --- We don't need QAOA_REPS anymore ---

# ----- Main Calculation Function -----
def main():
    """
    Calculates the H2 ground state energy using VQE with a UCCSD Ansatz.
    """
    print(f"H₂ Ground State Calculation for Basis: {BASIS_SET}")
    print(f"Using VQE with UCCSD Ansatz")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Define the molecule's geometry as a string
    atom_str = f"H 0 0 0; H 0 0 {BOND_LENGTH}"

    # --- 1. Set up the Chemical Problem ---
    start_time = time.time()
    
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET)
    problem = driver.run()
    
    mapper = JordanWignerMapper()
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    print(f"Running on full {qubit_op.num_qubits}-qubit Hamiltonian (Z2 symmetries NOT applied).")
    print("-" * 50)
    
    # --- 3. Set up VQE components with UCCSD ---
    optimizer = COBYLA(maxiter=1000)
    estimator = Estimator()
    
    # --- Create the UCCSD Ansatz ---
    # We need to get the number of particles and orbitals from the problem
    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals
    
    # Create the UCCSD circuit
    ansatz = UCCSD(
        num_spatial_orbitals,
        num_particles,
        mapper
    )
    
    # UCCSD is best started from the Hartree-Fock state (all parameters 0)
    # so we don't need a random initial point.
    vqe_instance = VQE(estimator,
                         ansatz,
                         optimizer) 
    
    # --- 4. Run the Algorithm ---
    print("Running VQE...")
    result_vqe = vqe_instance.compute_minimum_eigenvalue(
        qubit_op
    )
    
    # --- 5. Interpret and Display Results ---
    final_result = problem.interpret(result_vqe)
    final_energy = final_result.total_energies[0]
    
    print(f"\nCalculation finished in {time.time() - start_time:.2f}s\n")
    print("="*50)
    print("           Final Energy Summary")
    print("="*50)
    print(f"  VQE (UCCSD) Energy: {final_energy} Hartree")
    print(f"  Reference FCI (Exact) Energy: -1.137270174660903 Hartree")
    print("-" * 50)
    print(f"  Error (VQE vs FCI):  {abs(final_energy - (-1.137270174660903))} Hartree")
    print("="*50)


if __name__ == "__main__":
    main()

H₂ Ground State Calculation for Basis: sto-3g
Using VQE with UCCSD Ansatz
Started at: 2025-11-03 10:43:47

Running on full 4-qubit Hamiltonian (Z2 symmetries NOT applied).
--------------------------------------------------
Running VQE...


/tmp/ipykernel_25874/3767507981.py:50: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()



Calculation finished in 1.12s

           Final Energy Summary
  VQE (UCCSD) Energy: 0.7137539936876112 Hartree
  Reference FCI (Exact) Energy: -1.137270174660903 Hartree
--------------------------------------------------
  Error (VQE vs FCI):  1.8510241683485142 Hartree


H₂ Ground State Calculation for Basis: sto-3g
Using VQE with QAOAAnsatz (p=7)
Started at: 2025-11-03 10:46:30

Running on full 4-qubit Hamiltonian (Z2 symmetries NOT applied).
--------------------------------------------------
Running VQE...
/tmp/ipykernel_25874/3798246587.py:54: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()

Calculation finished in 73.31s

==================================================
           Final Energy Summary
==================================================
  VQE (QAOAAnsatz, p=7) Energy: -1.1103423037505213 Hartree
  Reference FCI (Exact) Energy: -1.137270174660903 Hartree
--------------------------------------------------
  Error (VQE vs FCI):  0.026927870910381735 Hartree
==================================================

H₂ Ground State Calculation for Basis: sto-3g
Using VQE with QAOAAnsatz (p=5)
Started at: 2025-11-03 10:44:40

Running on full 4-qubit Hamiltonian (Z2 symmetries NOT applied).
--------------------------------------------------
Running VQE...
/tmp/ipykernel_25874/2891662270.py:54: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()

Calculation finished in 59.42s

==================================================
           Final Energy Summary
==================================================
  VQE (QAOAAnsatz, p=5) Energy: -1.0980190304490605 Hartree
  Reference FCI (Exact) Energy: -1.137270174660903 Hartree
--------------------------------------------------
  Error (VQE vs FCI):  0.039251144211842615 Hartree
==================================================

In [14]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
# --- FIX 1: Import TaperedQubitMapper ---
from qiskit_nature.second_q.mappers import JordanWignerMapper, TaperedQubitMapper

# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Estimator, Sampler
# --- FIX 1: Remove invalid Z2Symmetries import ---
# from qiskit.quantum_info import Z2Symmetries # This class is no longer used

# ----- Configuration -----
BASIS_SET = "sto-3g"
BOND_LENGTH = 0.7414  # Angstrom
QAOA_REPS = 2         # QAOA repetitions (p-value)

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of H2 using three different methods:
    UHF, FCI, and QAOA.
    """
    print(f"H₂ Ground State Calculation Comparison for Basis: {BASIS_SET}")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Define the molecule's geometry as a string
    atom_str = f"H 0 0 0; H 0 0 {BOND_LENGTH}"

    # Dictionary to store results
    results = {}

    # --- 1. Classical Calculations (UHF and FCI) ---
    print("--- Running Classical Calculations ---")
    start_time = time.time()
    mol = gto.M(atom=atom_str, basis=BASIS_SET, verbose=0)
    
    # UHF Calculation
    mf_uhf = scf.UHF(mol).run()
    results['UHF'] = mf_uhf.e_tot
    print(f"UHF Energy: {results['UHF']} Hartree")
    
    # FCI Calculation
    mf_rhf = scf.RHF(mol).run()
    results['FCI'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    print(f"FCI (Exact) Energy: {results['FCI']} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Quantum Calculation (QAOA) ---
    print(f"--- Running QAOA Calculation (p={QAOA_REPS}) ---")
    start_time = time.time()
    
    # Set up the chemical problem for Qiskit
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET)
    problem = driver.run()
    
    # --- FIX 1: Use TaperedQubitMapper for modern Z2 symmetry reduction ---
    
    # 1. Create the base mapper
    jw_mapper = JordanWignerMapper()
    
    # 2. Get the full (untapered) qubit count
    full_qubit_op = jw_mapper.map(problem.hamiltonian.second_q_op())
    original_qubits = full_qubit_op.num_qubits

    # 3. Create a TaperedQubitMapper. This reads the symmetries
    #    directly from the problem object.
    tapered_mapper = TaperedQubitMapper(
        jw_mapper, 
        problem.symmetry_sector_locator(problem.z2_symmetries)
    )
    
    # 4. Map the Hamiltonian, which will now be tapered.
    qubit_op = tapered_mapper.map(problem.hamiltonian.second_q_op())
    
    print(f"Applied Z2 symmetries via TaperedQubitMapper. Reduced qubits from {original_qubits} to {qubit_op.num_qubits}.")
    # --- End of FIX 1 ---
    

    # Set up QAOA components
    optimizer = COBYLA(maxiter=1000)
    
    # Use the V1 Estimator (as V1 QAOA requires it)
    # The deprecation warning is expected, as we are mixing V1 algorithms
    # with a V2 (Qiskit 1.0+) environment.
    estimator = Estimator()
    sampler = Sampler()
    
    # QAOA requires an initial point with 2*reps parameters
    init_point = np.random.random(2 * QAOA_REPS) * 0.1
    
    # Instantiate QAOA
    qaoa_instance = QAOA(sampler,
                         optimizer=optimizer,
                         reps=QAOA_REPS,
                         initial_point=init_point)
    
    # --- FIX 2: Set the PUBLIC 'estimator' property ---
    # This prevents SamplingVQE from defaulting to _DiagonalEstimator
    # and fixes the "ValueError: The observable must be diagonal."
    qaoa_instance.estimator = estimator
    
    # Run QAOA
    result_qaoa = qaoa_instance.compute_minimum_eigenvalue(qubit_op)
    
    # Interpret the result to get the final energy
    # We must use the *tapered_mapper* to interpret the tapered result
    results['QAOA'] = problem.interpret(result_qaoa, tapered_mapper).total_energies[0]
    
    print(f"QAOA (p={QAOA_REPS}) Energy: {results['QAOA']} Hartree")
    print(f"QAOA finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("                     Final Energy Summary")
    print("="*50)
    print(f"  UHF Energy:         {results['UHF']} Hartree")
    print(f"  QAOA (p={QAOA_REPS}) Energy: {results['QAOA']} Hartree")
    print(f"  FCI (Exact) Energy: {results['FCI']} Hartree")
    print("-" * 50)
    # Compare quantum result to the exact FCI energy
    print(f"  Error (QAOA vs FCI):  {abs(results['QAOA'] - results['FCI'])} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

H₂ Ground State Calculation Comparison for Basis: sto-3g
Started at: 2025-11-03 11:09:58

--- Running Classical Calculations ---
UHF Energy: -1.1166843870853413 Hartree
FCI (Exact) Energy: -1.137270174660903 Hartree
Classical calculations finished in 0.13s

--- Running QAOA Calculation (p=2) ---


AttributeError: 'ElectronicStructureProblem' object has no attribute 'symmetry_sector_locator'